In [1]:
import pandas as pd
import os
import seaborn as sb
import datetime
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
import sklearn as sk

# Load Data

In [2]:
client_df = pd.read_csv('.\\ficheiros_competicao_dev\\client.csv', sep=';', low_memory=False)

account_df = pd.read_csv('.\\ficheiros_competicao_dev\\account.csv', sep=';', low_memory=False)

trans_dev_df = pd.read_csv('.\\ficheiros_competicao_dev\\trans_dev.csv', sep=';', low_memory=False)

loan_dev_df = pd.read_csv('.\\ficheiros_competicao_dev\\loan_dev.csv', sep=';', low_memory=False)

card_dev_df = pd.read_csv('.\\ficheiros_competicao_dev\\card_dev.csv', sep=';', low_memory=False)

disp_df = pd.read_csv('.\\ficheiros_competicao_dev\\disp.csv', sep=';', low_memory=False)

district_df = pd.read_csv('.\\ficheiros_competicao_dev\\district.csv', sep=';', low_memory=False, na_values='?')

FileNotFoundError: [Errno 2] No such file or directory: '.\\ficheiros_competicao_dev\\client.csv'

# Domain Analysis

## Business Understanding

### End User Requirements

The end user requires a system to determine which clients are and are not capable of paying of the loans they seek to make of the end user.

### Business Goals

In this problem the positive case is a client that cannot pay a loan. The business goal is to create a system to:
- Reduce the amount of loan attributions to clients who will default on the loan
- Mantain the amount of loan attributions to clients who can fulfill the loan

### Data Mining Goals

From the business goals we can determine that the goal of the model is to avoid granting a loan to a client who cannot pay it back, minimize false negatives so we must optimze for recall.

## Data Understanding

### Loans

In [ ]:
len(loan_dev_df[loan_dev_df['status'] == 1]) / len(loan_dev_df) * 100

Around 86% of loans in the dataset have been payed off, so accuracy is not the best measure to optimize for.

In [ ]:
sb.violinplot(x='status', y='amount', data=loan_dev_df, hue='status')

We can see a larger amount of the loans over 100,000 are not payed off.

# Data Treatment

## Whitespace Removal

In [ ]:
district_df.rename(columns=lambda x: x.strip(), inplace=True)

## Missing Values

### Acount

In [ ]:
account_df.isnull().sum()

No missing values on **account_df**

### Client

In [ ]:
client_df.isnull().sum()

No missing values on **client_df**

### Loans

In [ ]:
loan_dev_df.isnull().sum()

No missing values on **loan_dev_df**

### Transactions

In [ ]:
trans_dev_df.isnull().sum()

There are missing values on **trans_dev_df**

#### Operation

In [ ]:
trans_dev_df['operation'].value_counts()

In [ ]:
len(trans_dev_df[trans_dev_df['operation'].isnull()]) / len(trans_dev_df['operation']) * 100

There is a significant number of null values in the operations column. These will be replaced by 'N/A'.

In [ ]:
trans_dev_df['operation'].fillna('N/A', inplace=True)

In [ ]:
trans_dev_df['operation'].value_counts()

#### K Symbol

In [ ]:
trans_dev_df['k_symbol'].value_counts()

There are many empty string values. It is assumed these mean the transaction type wasn't registered and they will be treated the same as null values.

In [ ]:
def remove_empty_values_k_symbol(k):
    if isinstance(k, float):
        return'N/A'
    elif isinstance(k, str) and k.strip() == '':
        return 'N/A'
    else:
        return k

trans_dev_df['k_symbol'] = trans_dev_df['k_symbol'].apply(remove_empty_values_k_symbol)

trans_dev_df['k_symbol'].value_counts()

#### Bank

In [ ]:
trans_dev_df['bank'].value_counts()

There aren't any empty strings. Null values will be replaced by unknown.

In [ ]:
trans_dev_df['bank'].fillna('Unknown', inplace=True)

In [ ]:
trans_dev_df['bank'].value_counts()

#### Account

In [ ]:
trans_dev_df['account'].value_counts()

There are many transactions to an account **0**. This will be treated as a transaction to an unknown account.

In [ ]:
trans_dev_df['account'].fillna(0, inplace=True)

In [ ]:
trans_dev_df['account'].value_counts()

### Cards

In [ ]:
card_dev_df.isnull().sum()

There are no missing values in **card_dev_df**

### Disposition

In [ ]:
disp_df.isnull().sum()

There are no missing values in **disp_df**

### District

In [ ]:
district_df.isnull().sum()

There are missing values in **district_df**

In [ ]:
district_df[district_df['unemploymant rate \'95'].isnull()]

Both null values com from the district of Jesenik.

We will replace the null values by finding the mean values for the region in 1995 and 1996 and inferring the 1995 values for Jesenik from the 1996 values.

In [ ]:
unemp_rate_96_mean = district_df[district_df['region'] == 'north Moravia']['unemploymant rate \'96'].median()

unemp_rate_95_mean = district_df[district_df['region'] == 'north Moravia']['unemploymant rate \'95'].median()

unemp_rate_96_jes = district_df.iloc[68]['unemploymant rate \'96']

predicted_unemp_rate_95_jes = (unemp_rate_95_mean/unemp_rate_96_mean) * unemp_rate_96_jes

district_df['unemploymant rate \'95'].fillna(predicted_unemp_rate_95_jes, inplace=True)

crimes_96_mean = district_df[district_df['region'] == 'north Moravia']['no. of commited crimes \'96'].median()

crimes_95_mean = district_df[district_df['region'] == 'north Moravia']['no. of commited crimes \'95'].median()

crimes_96_jes = district_df.iloc[68]['no. of commited crimes \'96']

predicted_crimes_95_jes = (crimes_95_mean/crimes_96_mean) * crimes_96_jes

district_df['no. of commited crimes \'95'].fillna(predicted_crimes_95_jes, inplace=True)

## Date Functions

In [ ]:
# Format birth number to date (DD-MM-YY) <- may need to change formatting for algorithms
def get_formatted_date(date_number):
    date_number_string = str(date_number)
    return date_number_string[4:6] + '/' + str(int(date_number_string[2:4]) % 50) + '/' + '19' + date_number_string[0:2]

# Get client sex from birth number (MM > 50 => sex == 'F')
def get_client_sex_from_birth_number(date_number):
    return 'F' if int(str(date_number)[2:4]) >= 51 else 'M'

def get_season_from_Date(date_string):
    date_string_str = str(date_string)
    day = int(date_string_str[8:10])
    month = int(date_string_str[5:7])
    year = int(date_string_str[0:4])

    winterStart = datetime.datetime(year, 12, 22)
    springStart = datetime.datetime(year, 3, 20)
    summerStart = datetime.datetime(year, 6, 21)
    autumnStart = datetime.datetime(year, 9, 22)

    if ((month <= 3 and day <= 20) or (month >= 12 and day > 22)):
        return "winter"
    elif (date_string < summerStart):
        return "spring"
    elif (date_string < autumnStart) : 
        return "summer"
    elif (date_string < winterStart) :
        return "autumn"

    return "FAILEDSEASON"

## Format dates and determine client sex

In [ ]:
# Format client birthday and determine sex

client_df['birthday'] = pd.to_datetime(client_df['birth_number'].apply(get_formatted_date), infer_datetime_format=True)

client_df['sex'] = client_df['birth_number'].apply(get_client_sex_from_birth_number)

client_df = client_df.drop(columns=['birth_number'])

# Format other dates

account_df['acc_creation_date'] = pd.to_datetime(account_df['date'].apply(get_formatted_date), infer_datetime_format=True)

account_df = account_df.drop(columns=['date'])

trans_dev_df['trans_date'] = pd.to_datetime(trans_dev_df['date'].apply(get_formatted_date), infer_datetime_format=True)

trans_dev_df = trans_dev_df.drop(columns=['date'])

loan_dev_df['date'] = pd.to_datetime(loan_dev_df['date'].apply(get_formatted_date), infer_datetime_format=True)

card_dev_df['issued'] = pd.to_datetime(card_dev_df['issued'].apply(get_formatted_date), infer_datetime_format=True)

# Join Data

Data must all be displayed in one dataset.

## Join Account and Disposition

In [ ]:
joined_df = account_df.merge(disp_df, on='account_id', how='inner', suffixes=['', '_disp'])

joined_df.rename(columns={
    'type': 'account_type',
    'frequency': 'issuance_freq'
}, inplace=True)

# Determine if account is shared or not
owner_number_account = joined_df['account_id'].value_counts()

joined_df['shared'] = joined_df.apply(lambda row: 1 if owner_number_account[row['account_id']] > 1 else 0, axis=1)

# Drop rows with disponents so there are no duplicated account rows
joined_df.drop(joined_df[joined_df['account_type'] == 'DISPONENT'].index, inplace=True)

# Drop account_type and disp_id column
joined_df.drop(columns=['account_type'], inplace=True)

joined_df.head()

## Join Clients

In [ ]:
joined_df = joined_df.merge(client_df, on='client_id', how='left', suffixes=['', '_client'])

joined_df.drop(columns=['client_id'], inplace=True)

joined_df.head()

## Join District

There are currently two District ids in the dataset for each row. We are going to join on the client's district id since we surmise this information will be more relevant to wheter they pay off the loan or not than the district the account was registered on.

In [ ]:
joined_df = joined_df.merge(district_df, left_on='district_id_client', right_on='code', how='inner', suffixes=['', '_district'])

joined_df.rename(columns={
    'code': 'district_code',
    'name': 'district_name'
}, inplace=True)

joined_df.drop(columns=['district_id', 'district_id_client'], inplace=True)

joined_df.head()

## Join Loan

In [ ]:
joined_df = joined_df.merge(loan_dev_df, on='account_id', how='right', suffixes=['', '_loan'])

joined_df.drop(columns=['loan_id'], inplace=True)

joined_df.rename(columns={
    'date': 'loan_date'
}, inplace=True)

# Add seasons
joined_df['season_on_loan'] = joined_df['loan_date'].apply(get_season_from_Date)

joined_df.head(100)

## Determine account owner age on loan request

In [ ]:
joined_df['age_on_loan_request'] = joined_df.apply(lambda row: (row['loan_date'] - row['birthday'])/np.timedelta64(1, 'Y'), axis=1)

joined_df.drop(columns=['birthday'], inplace=True)

joined_df.head()

## Join Cards

There are a lot of clients without cards so simply joining the datasets would result in a large amount of null values. Instead we will record the card score attached to each account giving more weight to *classic* and *gold* cards. This is done beacause to have access to better credit cards the client must have a history of making and paying of loans in time.

In [ ]:
card_disp_df = card_dev_df.merge(disp_df, on='disp_id', how='inner', suffixes=['_card', '_disp'])

cards_per_user_df = card_disp_df.groupby(['account_id', 'type_card']).size().unstack(fill_value=0)

joined_df = joined_df.merge(cards_per_user_df, on='account_id', how='left', suffixes=['', ''])

joined_df.fillna(0, inplace=True)

joined_df['card_score'] = (joined_df['junior'] + joined_df['classic'] * 3 + joined_df['gold'] * 10).astype(int)

joined_df.drop(columns=['junior', 'classic', 'gold'], inplace=True)

joined_df.head()

## Join Transactions

In [ ]:
from dateutil.relativedelta import relativedelta

account_balance_df = trans_dev_df[['account_id', 'balance', 'trans_date']]

def find_balance_at_date(joined_df_row):
    # Get all balance information for account
    account_balances = account_balance_df[account_balance_df['account_id'] == joined_df_row['account_id']]
    
    # Get balance at loan date
    account_balances_at_loan_date = account_balances.copy()

    account_balances_at_loan_date['days_since_trans'] = account_balances_at_loan_date.apply(lambda row: (joined_df_row['loan_date'] - row['trans_date']), axis=1)

    account_balances_at_loan_date = account_balances_at_loan_date[account_balances_at_loan_date['days_since_trans'] >= pd.Timedelta(0)]

    account_balances_at_loan_date = account_balances_at_loan_date[account_balances_at_loan_date['days_since_trans'] == account_balances_at_loan_date['days_since_trans'].min()]


    # Get balance three months before loan date
    three_months_ago = joined_df_row['loan_date'] - relativedelta(months=3)

    account_balances_three_months_ago = account_balances.copy()

    account_balances_three_months_ago['days_since_trans'] = account_balances_three_months_ago.apply(lambda row: (three_months_ago - row['trans_date']), axis=1)

    account_balances_three_months_ago = account_balances_three_months_ago[account_balances_three_months_ago['days_since_trans'] >= pd.Timedelta(0)]

    account_balances_three_months_ago = account_balances_three_months_ago[account_balances_three_months_ago['days_since_trans'] == account_balances_three_months_ago['days_since_trans'].min()]

    # Get balance six months before loan date
    six_months_ago = joined_df_row['loan_date'] - relativedelta(months=6)

    account_balances_six_months_ago = account_balances.copy()

    account_balances_six_months_ago['days_since_trans'] = account_balances_six_months_ago.apply(lambda row: (six_months_ago - row['trans_date']), axis=1)

    account_balances_six_months_ago = account_balances_six_months_ago[account_balances_six_months_ago['days_since_trans'] >= pd.Timedelta(0)]

    account_balances_six_months_ago = account_balances_six_months_ago[account_balances_six_months_ago['days_since_trans'] == account_balances_six_months_ago['days_since_trans'].min()]


    joined_df_row['balance_at_loan'] = account_balances_at_loan_date.iloc[0]['balance'] if len(account_balances_at_loan_date.index) >= 1 else 0
    joined_df_row['balance_three_months_before'] = account_balances_three_months_ago.iloc[0]['balance'] if len(account_balances_three_months_ago.index) >= 1 else 0
    joined_df_row['balance_six_months_before'] = account_balances_six_months_ago.iloc[0]['balance'] if len(account_balances_six_months_ago.index) >= 1 else 0

    return joined_df_row[['balance_at_loan', 'balance_three_months_before', 'balance_six_months_before']]

joined_df[['balance_at_loan', 'balance_three_months_before', 'balance_six_months_before']] = joined_df.apply(find_balance_at_date, axis=1)

joined_df.head()

## Outliers

In [ ]:
joined_df.describe()[['no. of commited crimes \'95', 'age_on_loan_request', 'duration', 'payments']]

In [ ]:
import plotly.express as px

fig = px.histogram(joined_df, x='no. of commited crimes \'95')
fig.show()

In [ ]:
def find_outliers_IQR(df):

   q1=df.quantile(0.25)

   q3=df.quantile(0.75)

   IQR=q3-q1

   outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]

   return outliers

outliers = find_outliers_IQR(joined_df['no. of commited crimes \'95'])

print("number of outliers: " + str(len(outliers)))

print("max outlier value: " + str(outliers.max()))

print("min outlier value: " + str(outliers.min()))

### Remove underage users

In [ ]:
underage = joined_df.loc[joined_df['age_on_loan_request']<18]

joined_df.drop(underage.index, inplace=True)

joined_df.describe()[['age_on_loan_request']]


# Split into age brackets

In [ ]:
import plotly.express as px

fig = px.histogram(joined_df, x='age_on_loan_request')
fig.show()

In [ ]:
def get_age_bracket(age):
    if age >= 18 and age <= 24:
        return '18-24'
    elif age >= 25 and age <= 34:
        return '25-34'
    elif age>= 35 and age <= 44:
        return '35-44'
    elif age >= 45 and age <= 54:
        return '45-54'
    elif age >= 55 and age <= 64:
        return '55-64'
    elif age >= 65:
        return '65+'

joined_df['age_bracket'] = joined_df['age_on_loan_request'].apply(get_age_bracket)

joined_df.head()

## Set Correct Data Types

In [ ]:
joined_df['shared'] = joined_df['shared'].astype('bool')

In [ ]:
joined_df['status'] = joined_df['status'].apply(lambda x: 1 if x == 1 else 0)
joined_df['status'] = joined_df['status'].astype('bool')

In [ ]:
joined_df.drop(columns=['district_name', 'account_id', 'disp_id', 'age_on_loan_request'], inplace=True)

# Feature Selection

In [ ]:
features = [
    'shared',
    'no. of municipalities with inhabitants < 499',
    'no. of municipalities with inhabitants 500-1999',
    'no. of municipalities with inhabitants 2000-9999',
    'no. of municipalities with inhabitants >10000', 
    'no. of cities',
    'ratio of urban inhabitants', 
    'average salary', 
    'unemploymant rate \'96', 
    'no. of enterpreneurs per 1000 inhabitants',
    'amount', 
    'duration', 
    'payments',
    'age_bracket',
    'card_score', 
    'balance_at_loan',
    'balance_three_months_before',
    'balance_six_months_before',
    'issuance_freq',
    'sex', 
    'district_code',
    'region',
    'season_on_loan'
    ]
    
target = 'status'

### Remove Categorical Values

In [ ]:
cat = ['sex', 'region', 'issuance_freq', 'season_on_loan', 'age_bracket']

joined_df[cat] = joined_df[cat].astype('category')

joined_df[cat] = joined_df[cat].apply(lambda x : x.cat.codes)

In [ ]:
corr_matrix = joined_df[features].corr(method='spearman')

mask = np.zeros_like(corr_matrix, dtype=bool)
mask[np.triu_indices_from(mask)] = True

plt.figure(figsize=(20, 20))
plt.title('Correlation Matrix', fontsize=25)
_ = sb.heatmap(corr_matrix, mask=mask, cmap='coolwarm', annot=True, fmt='.2f', linewidths=2)

plt.show()

# ANOVA

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

def do_anova(cols, k="all"):
    best_features = SelectKBest(score_func=f_classif, k=k)
    df_cut = joined_df[cols]
    fit = best_features.fit(df_cut, joined_df["status"])
    dfscores = pd.DataFrame(fit.scores_)
    dfcolumns = pd.DataFrame(df_cut.columns)

    # concat 2 dataFrame for better visualization
    feature_score = pd.concat([dfcolumns, dfscores], axis=1)
    feature_score.columns = ["Features", "Score"]
    ret = feature_score.sort_values(by="Score", ascending=False)
    return ret.head(k) if k != "all" else ret

In [ ]:
do_anova(features)

In [ ]:
selected_features = [
    'shared',
    'balance_at_loan',
    'payments',
    'amount',
    'balance_six_months_before',
    'balance_three_months_before',
    'no. of municipalities with inhabitants < 499',
    'no. of cities',
    'season_on_loan',
    'age_bracket'
    ]

# Algorithms

In [ ]:
print(joined_df.columns)

In [ ]:
print(joined_df.dtypes)

In [ ]:
X = joined_df[selected_features]
y = joined_df[target]

In [ ]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y) # 70% training and 30% test


### Normalization
Some of the algorithms we plan on using (KNN and SVM) require the data to be standardized. To do so, we used a StandardScaler from SciKit Learn's preprocessing library.

In [ ]:
scaler = sk.preprocessing.StandardScaler()

scaler.fit(X_train)
x_train = scaler.fit_transform(X_train)
x_test = scaler.fit_transform(X_test)

## Descision Tree

In [ ]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [ ]:
print("Recall:",metrics.recall_score(y_test, y_pred))

In [ ]:
print("Precision:", metrics.precision_score(y_test, y_pred))

In [ ]:
from sklearn import tree

fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(clf, 
                   feature_names=selected_features,  
                   class_names=target,
                   filled=True)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold

clf = RandomForestClassifier(n_jobs=-1)

param_grid = {
    'min_samples_split': [3, 5, 10], 
    'n_estimators' : [100, 300],
    'max_depth': [3, 5, 15, 25],
    'max_features': [3, 5, 10, 20]
}

scorers = {
    'precision_score': make_scorer(precision_score),
    'recall_score': make_scorer(recall_score),
    'accuracy_score': make_scorer(accuracy_score)
}

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Recall:",metrics.recall_score(y_test, y_pred))

# SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE

oversample = SMOTE()
X_train_smote, y_train_smote = oversample.fit_resample(X_train, y_train)

rfc = RandomForestClassifier(n_jobs=-1)
rfc.fit(X_train_smote, y_train_smote)
y_pred_smote = rfc.predict(X_test)

print("Recall:",metrics.recall_score(y_test, y_pred_smote))

In [ ]:
len(X_train_smote)

## KNN

In [ ]:
k_range = [5, 10, 15, 20]
weight_options = ["uniform", "distance"]
leaf_size=[10,20,30,40,50,60]
p=[1,2,3,4,5] 
algorithm=['auto', 'ball_tree', 'kd_tree', 'brute']

parameter_grid = param_grid = dict(n_neighbors = k_range, weights = weight_options,algorithm= algorithm ,leaf_size = leaf_size , p =p)

knn = sk.neighbors.KNeighborsClassifier()

grid = sk.model_selection.GridSearchCV(knn, param_grid, cv = 10, scoring = 'accuracy')
grid.fit(X_train,y_train)


y_pred = grid.predict(X_test)
print(sk.metrics.accuracy_score(y_test, y_pred))
print("Recall: ",sk.metrics.recall_score(y_test, y_pred))
print("Precision: ", sk.metrics.precision_score(y_test, y_pred))